In [3]:
import json
import numpy as np
import trimesh
from utils.data_load import load_image, load_3d_model, visualize_data # our own utils

# Get main data

In [4]:
dir = "data/pix3d"

In [5]:
with open(f"{dir}/pix3d.json", "rb") as f:
    metadata = json.load(f)

In [6]:
imgs, masks, models = [], [], []
for i in range(len(metadata)):
    img_path = dir+'/'+metadata[i]["img"]
    mask_path = dir+'/'+metadata[i]["mask"]
    model_path = dir+'/'+metadata[i]["model"]

    # take actual img, mask and model
    img, mask = load_image(img_path, mask_path)
    model_img = load_3d_model(model_path)

    # combine all geometries, if this is a scene
    if isinstance(model_img, trimesh.Scene):
        model_img = model_img.dump(concatenate=True)
    else:
        model_img = model_img

    imgs.append(img)
    masks.append(mask)
    models.append(model_img)
    
    break

C:\Users\laxon\AppData\Local\Temp\ipykernel_2100\2954166473.py:13: DeprecationWarning: `Scene.dump(concatenate=True)` DEPRECATED FOR REMOVAL APRIL 2025: replace with `Scene.to_geometry()`
  model_img = model_img.dump(concatenate=True)
